In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import io
from datetime import datetime
import logging
import pandas as pd

<IPython.core.display.Javascript object>

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
# Define the bucket name and region
S3_BUCKET = os.getenv("S3_BUCKET")
S3_PREFIX = os.getenv("S3_PREFIX")
REGION = os.getenv("REGION")
SAGE_MAKER_LOCAL_ROLE = os.getenv("SAGE_MAKER_LOCAL_ROLE")
print(f"S3_BUCKET: {S3_BUCKET}")
print(f"REGION: {REGION}")
print(f"SAGE_MAKER_LOCAL_ROLE: {SAGE_MAKER_LOCAL_ROLE}")

S3_BUCKET: sgmkr-thangtran3112
REGION: us-west-2
SAGE_MAKER_LOCAL_ROLE: arn:aws:iam::654654352356:role/service-role/AmazonSageMaker-ExecutionRole-20250111T085887


<IPython.core.display.Javascript object>

In [4]:
import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

/home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/16/25 19:20:56] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=602073;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=31614;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thangtran3112/.config/sagemaker/config.yaml


<IPython.core.display.Javascript object>

In [5]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

<IPython.core.display.Javascript object>

In [6]:
region = sagemaker.Session().boto_region_name
print(region)

us-west-2


<IPython.core.display.Javascript object>

In [7]:
import os

if "SM_CURRENT_HOST" in os.environ:
  print("Running in SageMaker Studio")
  # only inside Sagemaker notebook Studio
  role_arn = sagemaker.get_execution_role()
else:
  print("Not running in SageMaker Studio. Using custom role for local computer")
  # in local computer, we will get it from environment variable
  role_arn = SAGE_MAKER_LOCAL_ROLE

print(role_arn)

Not running in SageMaker Studio. Using custom role for local computer
arn:aws:iam::654654352356:role/service-role/AmazonSageMaker-ExecutionRole-20250111T085887


<IPython.core.display.Javascript object>

In [8]:
!aws s3 ls {S3_BUCKET}/{S3_PREFIX}/

                           PRE data/
                           PRE model/


<IPython.core.display.Javascript object>

In [9]:
!aws s3 ls {S3_BUCKET}/{S3_PREFIX}/data/ --recursive

2025-01-11 11:34:19        900 iris/data/iris_test.csv
2025-01-11 11:34:19       1800 iris/data/iris_train.csv


<IPython.core.display.Javascript object>

In [10]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

<IPython.core.display.Javascript object>

In [11]:
train_ip = TrainingInput(
    "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, train_file), content_type="csv"
)
valid_ip = TrainingInput(
    "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, valid_file), content_type="csv"
)

<IPython.core.display.Javascript object>

In [12]:
model_op = "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, "model")

<IPython.core.display.Javascript object>

In [13]:
train_image_uri = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(train_image_uri)

[01/16/25 19:21:01] INFO     Ignoring unnecessary instance type: None.                            ]8;id=683751;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=93956;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/image_uris.py#528\528]8;;\

433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest


<IPython.core.display.Javascript object>

In [14]:
base_job_name = "iris-xgboost"

<IPython.core.display.Javascript object>

In [15]:
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
)

<IPython.core.display.Javascript object>

In [16]:
xgb_estimator.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

<IPython.core.display.Javascript object>

In [17]:
# xgb_estimator.set_hyperparameters(
#     num_class=3,
#     max_depth=5,
#     eta=0.2,
#     gamma=4,
#     min_child_weight=6,
#     subsample=0.7,
#     objective="multi:softmax",
#     num_round=10,
# )

<IPython.core.display.Javascript object>

In [18]:
job_name = "iris-xgboost-" + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2025-01-16-19-21-02


<IPython.core.display.Javascript object>

In [19]:
xgb_estimator.fit(
    {"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name
)

[01/16/25 19:21:02] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=667004;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=191143;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: iris-xgboost-2025-01-16-19-21-02      ]8;id=684997;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=756543;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-01-17 03:21:03 Starting - Starting the training job...
2025-01-17 03:21:17 Starting - Preparing the instances for training...
2025-01-17 03:21:44 Downloading - Downloading input data...
2025-01-17 03:22:14 Downloading - Downloading the training image...
2025-01-17 03:23:04 Training - Training image download completed. Training in progress...Arguments: train
[2025-01-17:03:23:16:INFO] Running standalone xgboost training.
[2025-01-17:03:23:16:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8443.07mb
[2025-01-17:03:23:16:INFO] Determined delimiter of CSV input is ','
[03:23:16] S3DistributionType set as FullyReplicated
[03:23:16] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-01-17:03:23:16:INFO] Determined delimiter of CSV input is ','
[03:23:16] S3DistributionType set as FullyReplicated
[03:23:16] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=c

<IPython.core.display.Javascript object>

In [20]:
!aws s3 ls {S3_BUCKET}/{S3_PREFIX}/model/

                           PRE iris-xgboost-2025-01-11-18-03-55/
                           PRE iris-xgboost-2025-01-11-19-59-53/
                           PRE iris-xgboost-2025-01-16-19-21-02/


<IPython.core.display.Javascript object>

### Inference 

In [21]:
from sagemaker.serializers import CSVSerializer

<IPython.core.display.Javascript object>

#### Deploy the model as an endpoint

In [22]:
type(xgb_estimator)

sagemaker.estimator.Estimator

<IPython.core.display.Javascript object>

In [23]:
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium", serializer=CSVSerializer()
)

[01/16/25 19:23:55] INFO     Creating model with name: iris-xgboost-2025-01-17-03-23-55-165         ]8;id=215807;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=94322;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=744073;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=570298;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\
                             iris-xgboost-2025-01-17-03-23-55-165                                                  

[01/16/25 19:23:56] INFO     Creating endpoint with name iris-xgboost-2025-01-17-03-23-55-165       ]8;id=807637;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=503198;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\

------------!

<IPython.core.display.Javascript object>

#### Predictor single record

In [24]:
xgb_predictor.predict("7.7, 3.0, 6.1, 2.3")

b'2.0'

<IPython.core.display.Javascript object>

#### Endpoint

In [25]:
endpoint_name = xgb_predictor.endpoint_name
print(endpoint_name)

iris-xgboost-2025-01-17-03-23-55-165


<IPython.core.display.Javascript object>

In [26]:
sgmkr_runtime = boto3.client("runtime.sagemaker")

<IPython.core.display.Javascript object>

#### Endpoint - One record

In [27]:
payload_csv_text = "7.7, 3.0, 6.1, 2.3"
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
response

{'ResponseMetadata': {'RequestId': '2950134d-4aba-4757-a561-a5bf4d73f10f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2950134d-4aba-4757-a561-a5bf4d73f10f',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Fri, 17 Jan 2025 03:30:29 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '3',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7873027bfe50>}

<IPython.core.display.Javascript object>

In [28]:
print(response["Body"].read().decode())

2.0


<IPython.core.display.Javascript object>

#### Endpoint - Multiple records

In [29]:
payload_csv_text = "7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1"

response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

2.0,2.0


<IPython.core.display.Javascript object>

#### Endpoint - Multiple records from a local file

In [30]:
csv_buffer = open("data/iris_infer.csv")
payload_csv_text = csv_buffer.read()
payload_csv_text

'6.3,2.8,5.1,1.5\n6.3,3.3,4.7,1.6\n5.0,3.4,1.5,0.2\n5.8,2.7,4.1,1.0\n7.3,2.9,6.3,1.8\n4.9,2.4,3.3,1.0\n5.7,2.8,4.5,1.3\n5.7,3.8,1.7,0.3\n5.6,3.0,4.5,1.5\n5.5,2.3,4.0,1.3\n4.4,3.2,1.3,0.2\n5.8,4.0,1.2,0.2\n5.1,3.3,1.7,0.5\n5.1,3.4,1.5,0.2\n5.4,3.7,1.5,0.2\n6.4,2.8,5.6,2.2\n6.0,3.0,4.8,1.8\n5.6,2.5,3.9,1.1\n7.7,2.8,6.7,2.0\n5.7,2.8,4.1,1.3\n6.5,3.0,5.2,2.0\n5.6,3.0,4.1,1.3\n4.7,3.2,1.3,0.2\n6.5,3.0,5.5,1.8\n4.6,3.6,1.0,0.2\n6.5,3.0,5.8,2.2\n6.7,3.1,5.6,2.4\n5.0,3.2,1.2,0.2\n5.4,3.4,1.7,0.2\n6.2,3.4,5.4,2.3\n6.4,2.7,5.3,1.9\n6.9,3.1,5.1,2.3\n5.1,3.7,1.5,0.4\n5.4,3.0,4.5,1.5\n5.2,3.4,1.4,0.2\n4.5,2.3,1.3,0.3\n6.7,3.0,5.2,2.3\n5.7,2.9,4.2,1.3\n6.7,3.0,5.0,1.7\n6.0,3.4,4.5,1.6\n6.1,2.9,4.7,1.4\n5.0,2.3,3.3,1.0\n4.4,3.0,1.3,0.2\n4.9,3.0,1.4,0.2\n6.1,2.6,5.6,1.4\n6.0,2.9,4.5,1.5\n6.7,2.5,5.8,1.8\n4.9,2.5,4.5,1.7\n6.4,3.2,4.5,1.5\n6.1,3.0,4.9,1.8\n'

<IPython.core.display.Javascript object>

In [33]:
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

2.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


<IPython.core.display.Javascript object>

#### Endpoint - Multiple records from a S3 file

In [36]:
infer_ip_s3_uri = "s3://{}/{}/{}".format(
    S3_BUCKET, S3_PREFIX, "batch_transform/iris_infer.csv"
)
s3_clnt = boto3.client("s3")

# Upload the local file to S3
s3_clnt.upload_file(
    Filename="data/iris_infer.csv",
    Bucket=S3_BUCKET,
    Key=f"{S3_PREFIX}/batch_transform/iris_infer.csv"
)

# Option 1: Read the CSV file from S3 using pandas
# payload_df = pd.read_csv(infer_ip_s3_uri)

# Option 2: Read the CSV file from S3 using AWS Data Wrangler
# payload_df = wr.s3.read_csv(path=infer_ip_s3_uri)

# Option 3: Read the CSV file from S3 using boto3
obj = s3_clnt.get_object(Bucket=S3_BUCKET, Key="iris/batch_transform/iris_infer.csv")
payload_df = pd.read_csv(obj["Body"])

csv_buffer = io.StringIO()
payload_df.to_csv(csv_buffer, header=None, index=None)
payload_csv_text = csv_buffer.getvalue()

response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


<IPython.core.display.Javascript object>

#### Delete the endpoint

In [37]:
sgmkr_clnt = boto3.client("sagemaker")

<IPython.core.display.Javascript object>

In [38]:
sgmkr_clnt.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '221ec3d7-2389-45ac-bec9-b3e468baf9c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '221ec3d7-2389-45ac-bec9-b3e468baf9c4',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 17 Jan 2025 03:37:54 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

<IPython.core.display.Javascript object>

#### Batch Transform
* Transform a batch of inputs into a batch of outputs, instead of invoking APIs

In [39]:
batch_ip = "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, "batch_transform")
batch_op = "s3://{}/{}/{}".format(S3_BUCKET, S3_PREFIX, "batch_transform")

<IPython.core.display.Javascript object>

In [40]:
!aws s3 ls {batch_ip}/ --recursive

2025-01-16 19:37:41        800 iris/batch_transform/iris_infer.csv


<IPython.core.display.Javascript object>

In [41]:
# We can also use multiple instances, when we have large input dataset
transformer = xgb_estimator.transformer(
    instance_count=1, instance_type="ml.m4.xlarge", output_path=batch_op
)

[01/16/25 19:38:31] INFO     Creating model with name: iris-xgboost-2025-01-17-03-38-31-754         ]8;id=822055;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=132808;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\

<IPython.core.display.Javascript object>

Must increase the Sagemaker transform job quota for the corresponding instance, to run the following code

In [42]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

[01/16/25 19:38:39] INFO     Creating transform job with name: iris-xgboost-2025-01-17-03-38-39-744 ]8;id=989704;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=821712;file:///home/thangtran3112/aws-sage-maker/machine-learning/venv/lib/python3.12/site-packages/sagemaker/session.py#3951\3951]8;;\

................................Arguments: serve
[2025-01-17 03:44:14 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2025-01-17 03:44:14 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2025-01-17 03:44:14 +0000] [1] [INFO] Using worker: gevent
[2025-01-17 03:44:14 +0000] [21] [INFO] Booting worker with pid: 21
[2025-01-17 03:44:14 +0000] [22] [INFO] Booting worker with pid: 22
[2025-01-17 03:44:14 +0000] [23] [INFO] Booting worker with pid: 23
[2025-01-17 03:44:14 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.p

<IPython.core.display.Javascript object>

In [43]:
!aws s3 ls {S3_BUCKET}/{S3_PREFIX}/batch_transform/ --recursive

2025-01-16 19:37:41        800 iris/batch_transform/iris_infer.csv
2025-01-16 19:44:19        200 iris/batch_transform/iris_infer.csv.out


<IPython.core.display.Javascript object>

* The classification results will be stored in `iris/batch_transform/iris_infer.csv.out`

In [44]:
!aws s3 cp s3://{S3_BUCKET}/{S3_PREFIX}/batch_transform/iris_infer.csv.out .

download: s3://sgmkr-thangtran3112/iris/batch_transform/iris_infer.csv.out to ./iris_infer.csv.out


<IPython.core.display.Javascript object>

In [45]:
!head -n 5 iris_infer.csv.out

2.0
1.0
0.0
1.0
2.0


<IPython.core.display.Javascript object>